In [1]:
def read_file_as_df(file_name):
    import pandas as pd
    import csv

    import sys
    import pandas as pd

    maxInt = sys.maxsize

    while True:
        # decrease the maxInt value by factor 10
        # as long as the OverflowError occurs.

        try:
            csv.field_size_limit(maxInt)
            break
        except OverflowError:
            maxInt = int(maxInt/10)

    file = []
    col = []

    with open(file_name, encoding='latin-1') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        line_count = 0
        for row in csv_reader:
            if line_count==0:
                for r in row:
                    col.append(r)
                line_count+=1
            else:
                line = []
                for r in row:
                    line.append(r)
                file.append(line)
                line_count += 1


    df = pd.DataFrame(file, columns = col)
    return df

In [2]:
import nltk

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

import string
import pandas as pd

[nltk_data] Downloading package stopwords to /home/alice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def remove_stopwords(text):
    all_words = text.split(" ")
    clean_text = [i for i in all_words if i not in stopwords and i!=""]
    return " ".join(clean_text)

def clean_text(x):
  return " ".join(str(x.translate(str.maketrans('', '', string.punctuation))).split()).lower()

def get_age_class(age):
    if age == 'a17-30': return 0
    elif age == 'a31-42': return 1
    else: return 2

In [4]:
df = read_file_as_df('../data/e-SIC1BR.csv')

In [5]:
df.head()

,age-bracket,gender,education,profession,req-text,resp-text,resp-type,clarity,service,1funct-request,...,55work-response,56achieve-response,57leisure-response,58home-response,59money-response,60relig-response,61death-response,62assent-response,63nonfl-response,64filler-response
0,a31-42,m,e2,academy,"Prezados, Gostaria de solicitar informações so...","Não dispomos de vaga em aberto, para o cargo d...",Acesso Concedido,5,5,"0,4531",...,"0,0612","0,102","0,0204",0,0,0,0,0,0,0
1,a31-42,m,e2,academy,"Prezados, Gostaria de solicitar informações so...","Prezado cidadão, O quadro de referência de ser...",Acesso Concedido,5,5,"0,4531",...,"0,0394","0,0472","0,0157",0,"0,0079",0,0,0,"0,0079",0
2,a31-42,m,e2,academy,"Prezados, Gostaria de solicitar informações so...","Sr. JOMAR BORGES DOS SANTOS, boa tarde. Em res...",Acesso Concedido,5,5,"0,4531",...,"0,0541","0,0541",0,0,0,"0,027",0,0,"0,027",0
3,a31-42,m,e2,academy,"Prezados, Gostaria de solicitar informações so...","Prezado Jomar, Encaminho resposta à sua solici...",Acesso Concedido,5,5,"0,4531",...,"0,0549","0,0706","0,0196",0,"0,0275","0,0078",0,0,"0,0235",0
4,a43-xx,na,na,na,gostaria de saber se a empresa a qual min refi...,"Prezado(a) Senhor(a), Informamos que a Secreta...",Acesso Concedido,1,1,"0,4286",...,"0,0476","0,0476",0,0,"0,0476","0,0159",0,0,"0,0476",0


In [6]:
df = read_file_as_df('../data/e-SIC1BR.csv')
df = df[['age-bracket', 'req-text']]

df["req-text-clean"] = df['req-text'].apply(lambda x: clean_text(x))
df["req-text-clean"] = df['req-text-clean'].apply(lambda x: remove_stopwords(x))
df["Age"] = df['age-bracket'].apply(lambda x: get_age_class(x))

X = df["req-text-clean"]
y = df['Age']

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=115, stratify=y)
#df_train, df_test = train_test_split(df, test_size=0.2, random_state=289, stratify=y) 


In [6]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [7]:
train_df.head()

,Unnamed: 0,age-bracket,req-text,req-text-clean,Age
0,3415,a43-xx,Gostaria de adotar um casal de cacatuas de um ...,gostaria adotar casal cacatuas criador argenti...,2
1,24254,a43-xx,"Minha esposa ganhou neném, mas na situação de ...",esposa ganhou neném situação natmorto nasceu m...,2
2,269,a43-xx,Peço a informação referente ao nome de todos o...,peço informação referente nome todos candidato...,2
3,24435,a17-30,o quantitativo de cargos ocupados e vagos do c...,quantitativo cargos ocupados vagos cargo técni...,0
4,15926,a17-30,Sou Aluna da Academia de Policia Militar Costa...,aluna academia policia militar costa verde pmm...,0


In [8]:
train_df[train_df['req-text'].isna()]

,Unnamed: 0,age-bracket,req-text,req-text-clean,Age


In [9]:
train_df[train_df['req-text']=='nan']

,Unnamed: 0,age-bracket,req-text,req-text-clean,Age


In [10]:
train_df = train_df[train_df['req-text']!='nan']

In [13]:
train_df.to_csv('../data/train.csv', index=True)
test_df.to_csv('../data/test.csv', index=True)

In [11]:
unique_words = list(set(" ".join(train_df["req-text-clean"].to_list()).split()))
print(len(unique_words))
threshold = train_df.shape[0]*0.01
word_counts = train_df["req-text-clean"].str.split().explode().value_counts()
row_counts = pd.DataFrame({'word': word_counts.index, 'count': word_counts.values})
row_counts['count'] = row_counts['word'].apply(lambda word: train_df["req-text-clean"].str.contains(word, regex=False).sum())
selected_words = row_counts[row_counts['count']>threshold]['word'].to_list()

85317


In [11]:
selected_words = []

# open file and read the content in a list
with open(r'../data/one_percent_vocab.txt', 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]

        # add current item to the list
        selected_words.append(x)

# display list
print(len(selected_words))

2776


In [12]:
train_df.shape[0]

38207

In [13]:
len(selected_words)

2769

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

text = train_df["req-text-clean"].to_list()

vectorizer = TfidfVectorizer(vocabulary=selected_words)
train_matrix = vectorizer.fit_transform(text)
train_matrix = train_matrix.toarray()

tf_idf = pd.DataFrame(data=train_matrix, columns=selected_words)

X = tf_idf
y =  train_df['Age']

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 100],  # Regularization parameter
    'max_iter': [1000, 10000, -1], # Maximum number of iterations
    'gamma' : ['scale', 'auto']
}

svc = SVC(kernel='linear', decision_function_shape='ovr')
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2)

# Fit the model to the training data
grid_search.fit(X, y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


/home/alice/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END ..................C=0.1, gamma=scale, max_iter=1000; total time= 5.5min


/home/alice/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END ..................C=0.1, gamma=scale, max_iter=1000; total time= 5.4min


...

[CV] END ..................C=100, gamma=auto, max_iter=10000; total time=45.1min


/home/alice/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END ..................C=100, gamma=auto, max_iter=10000; total time=28.8min
[CV] END ....................C=100, gamma=auto, max_iter=-1; total time=391.2min
[CV] END ....................C=100, gamma=auto, max_iter=-1; total time=397.6min
[CV] END ....................C=100, gamma=auto, max_iter=-1; total time=351.9min
[CV] END ....................C=100, gamma=auto, max_iter=-1; total time=347.0min
[CV] END ....................C=100, gamma=auto, max_iter=-1; total time=343.7min


/home/alice/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


GridSearchCV(cv=5, estimator=SVC(kernel='linear'),
             param_grid={'C': [0.1, 1, 100], 'gamma': ['scale', 'auto'],
                         'max_iter': [1000, 10000, -1]},
             scoring='accuracy', verbose=2)

In [16]:
grid_search.best_estimator_, grid_search.best_score_

model = grid_search.best_estimator_

In [14]:
from sklearn.svm import SVC

svc = SVC(kernel='linear', decision_function_shape='ovr')

svc.fit(X, y)

SVC(kernel='linear')

In [15]:
tf_idf.columns

Index(['gostaria', 'informações', 'solicito', 'saber', 'sobre', 'informação',
       'federal', 'nº', 'dados', 'acesso',
       ...
       'gat', 'teme', 'rat', 'aus', 'sita', 'oper', 'retam', '2720', 'aposent',
       'atura'],
      dtype='object', length=2776)

In [17]:
len(svc.coef_[0])

2776

In [18]:
df_coef = pd.DataFrame(data=svc.coef_, columns=tf_idf.columns)

In [19]:
df_coef.head()

,gostaria,informações,solicito,saber,sobre,informação,federal,nº,dados,acesso,...,gat,teme,rat,aus,sita,oper,retam,2720,aposent,atura
0,2.089167,-0.002321,-1.584382,-0.758446,0.095227,-0.959611,-0.459024,0.797148,1.034735,1.553217,...,0.0,0.000000,-0.374782,0.431735,-0.062730,-0.590606,0.0,0.0,0.000000,0.0
1,3.964612,0.350427,0.813003,-0.649238,-0.634475,-0.616576,-0.045993,0.514047,1.484101,1.305485,...,0.0,-0.151046,0.000000,0.431735,-0.280554,0.000000,0.0,0.0,-0.158455,0.0
2,4.014491,0.681776,3.067699,-0.648275,-0.621924,0.184383,0.208208,0.109771,0.906476,0.496693,...,0.0,-0.242287,0.374782,0.000000,-0.072500,0.590606,0.0,0.0,0.000000,0.0


In [20]:
df_coef = df_coef.T

In [21]:
df_coef.head()

,0,1,2
gostaria,2.089167,3.964612,4.014491
informações,-0.002321,0.350427,0.681776
solicito,-1.584382,0.813003,3.067699
saber,-0.758446,-0.649238,-0.648275
sobre,0.095227,-0.634475,-0.621924


In [81]:
def select_abs(col, perc): # mesma quantidade de palavras positivas e negativas
  n = int(len(df_coef) * perc)

  df_coef_1 = df_coef.copy()
  df_coef_1 = df_coef_1[[col]]

  df_coef_1['abs_values'] = df_coef_1[col].abs()

  sorted_df = df_coef_1.sort_values(by='abs_values', ascending=False)

  return sorted_df.head(n)

coef1 = select_abs(0, 0.5)
coef2 = select_abs(1, 0.5)
coef3 = select_abs(2, 0.5)

In [45]:
def select_(col, perc): # mesma quantidade de palavras positivas e negativas
  df_coef_1 = df_coef.copy()
  df_coef_1 = df_coef_1[[col]]

  str_col = str(col)

  df_coef_1.rename(columns={col: str_col}, inplace=True)

  df_coef_1_pos = df_coef_1[df_coef_1[str_col]>0]
  df_coef_1_neg= df_coef_1[df_coef_1[str_col]<0]
  df_coef_1_zero = df_coef_1[df_coef_1[str_col]==0]

  df_coef_1_pos = df_coef_1_pos.sort_values(by=str_col, ascending=True)
  df_coef_1_neg = df_coef_1_neg.sort_values(by=str_col, key=lambda x: abs(x))

  pos_perc = df_coef_1_pos.head(int(len(df_coef_1_pos) * perc))
  neg_perc = df_coef_1_neg.head(int(len(df_coef_1_neg) * perc))

  indices_to_drop = pd.concat([pos_perc, neg_perc, df_coef_1_zero]).index

  df_filtered = df_coef_1.drop(indices_to_drop)

  return df_filtered.sort_values(by=str_col, ascending=False)

coef1 = select_(0, 0.5)
coef2 = select_(1, 0.5)
coef3 = select_(2, 0.5)

In [48]:
coef1.head(10), coef2.head(10), coef3.head(10)

(                        0  abs_values
 load            -8.264912    8.264912
 doa             -4.101723    4.101723
 antecipadamente -3.502502    3.502502
 senhores        -3.439903    3.439903
 fundamenta      -3.094204    3.094204
 zen             -3.035773    3.035773
 estudante        2.995601    2.995601
 85              -2.984326    2.984326
 senhor          -2.638421    2.638421
 licitações      -2.581998    2.581998,
                         1  abs_values
 senhores        -4.356004    4.356004
 gostaria         3.964612    3.964612
 ouvidor         -3.593157    3.593157
 antecipadamente -3.589851    3.589851
 olá              3.375449    3.375449
 políticas        3.284270    3.284270
 informar        -3.055556    3.055556
 requisito        2.933390    2.933390
 vem             -2.928960    2.928960
 prezadoa         2.865195    2.865195,
                     2  abs_values
 gostaria     4.014491    4.014491
 ouvidor     -3.764982    3.764982
 entro        3.380243    3.380243


In [49]:
coef1['0']['estudante'], coef2['1']['senhores'], coef3['2']['senhores']

(3.0679075460247596, -4.495259336530953, -2.6158072366238514)

In [82]:
train_df['glex1']=0
train_df['glex2']=0
train_df['glex3']=0

In [66]:
train_df.reset_index(inplace=True)

In [86]:
# dict 1
for index, row in train_df.iterrows():
    sum = 0
    text = row["req-text-clean"].split()
    for w in text:
      if w in coef1.index.tolist():
        sum += coef1[0][w]*tf_idf[w][index]
    sum = sum + svc.intercept_[0]
    train_df[f'glex1'][index] = sum

/tmp/ipykernel_14084/1904768421.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[f'glex1'][index] = sum


In [85]:
# dict 2
for index, row in train_df.iterrows():
    sum = 0
    text = row["req-text-clean"].split()
    for w in text:
      if w in coef2.index.tolist():
        sum += coef2[1][w]*tf_idf[w][index]
    sum = sum + svc.intercept_[1]
    train_df[f'glex2'][index] = sum

/tmp/ipykernel_14084/3476015007.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[f'glex2'][index] = sum


In [89]:
# dict 3
for index, row in train_df.iterrows():
    sum = 0
    text = row["req-text-clean"].split()
    for w in text:
      if w in coef3.index.tolist():
        sum += coef3[2][w]*tf_idf[w][index]
    sum = sum + svc.intercept_[2]
    train_df[f'glex3'][index] = sum

/tmp/ipykernel_14084/4254082410.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[f'glex3'][index] = sum


In [90]:
train_df.groupby(['Age'])[['glex1', 'glex2', 'glex3']].describe().T

Age                     0             1             2
glex1 count  13415.000000  13580.000000  11212.000000
      mean       0.799643     -0.623071     -0.324490
      std        1.801296      2.054373      1.673470
      min      -25.141920    -23.948917    -27.249723
      25%       -0.073831     -1.331008     -1.149337
      50%        0.762829     -0.518761     -0.318625
      75%        1.473404      0.347711      0.520798
      max       43.967773     38.959068     19.052540
glex2 count  13415.000000  13580.000000  11212.000000
      mean       1.314676      0.496910     -0.519929
      std        2.013420      2.075745      1.744587
      min      -11.878998    -19.497586    -15.329909
      25%        0.303125     -0.555647     -1.335666
      50%        1.129684      0.417102     -0.616481
      75%        2.083480      1.496452      0.344194
      max       63.321329     27.772724     14.500883
glex3 count  13415.000000  13580.000000  11212.000000
      mean       0.822228      1.221823     -0.401759
      std        1.819496      2.255569      1.583413
      min      -12.163535    -27.918248    -14.169919
      25%       -0.104024      0.079459     -1.169784
      50%        0.730993      0.967015     -0.441989
      75%        1.583426      1.871338      0.422713
      max       47.457644     41.687685     14.091865

In [32]:
train_df['c'] = -1

for index, row in train_df.iterrows():
  if row['glex1'] > 0 and row['glex2'] > 0 and row['glex3'] > 0:
    train_df['c'][index] = 0
  elif row['glex1'] < 0 and row['glex2'] > 0 and row['glex3'] > 0:
    train_df['c'][index] = 1
  elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < 0:
    train_df['c'][index] = 2

/tmp/ipykernel_14084/4284971904.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['c'][index] = 1
/tmp/ipykernel_14084/4284971904.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['c'][index] = 0
/tmp/ipykernel_14084/4284971904.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['c'][index] = 2


KeyboardInterrupt: 

In [77]:
print(f'Instâncias totais: {train_df.shape[0]}')
print(f'Instâncias classificadas: {train_df[train_df.c!=-1].shape[0]}')
print(f'Porcentagem classificadas: {train_df[train_df.c!=-1].shape[0]/train_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {train_df[(train_df.c!=-1) & (train_df.c==train_df.Age)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {train_df[(train_df.c!=-1) & (train_df.c==train_df.Age)].shape[0]/train_df[train_df.c!=-1].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {train_df[(train_df.c!=-1) & (train_df.c==train_df.Age)].shape[0]/train_df.shape[0]}')

# as palavras não esto conseguindo separar bem (1% muito, restringir menos)

Instâncias totais: 38207
Instâncias classificadas: 25742
Porcentagem classificadas: 0.6737508833459838
Instâncias classificadas corretamente: 15449
Porcentagem classificadas corretamente: 0.6001476186776474
Porcentagem classificadas corretamente das totais: 0.40434998822205354


In [91]:
test_matrix = vectorizer.transform(test_df["req-text-clean"].to_list())
test_matrix = test_matrix.toarray()

tf_idf_test = pd.DataFrame(data=test_matrix, columns=selected_words)

#test_df.reset_index(inplace=True)

test_df['glex1']=0
test_df['glex2']=0
test_df['glex3']=0

# dict 1
for index, row in test_df.iterrows():
    sum = 0
    text = row["req-text-clean"].split()
    for w in text:
      if w in coef1.index.tolist():
        sum += coef1[0][w]*tf_idf_test[w][index]
    sum = sum + svc.intercept_[0]
    test_df[f'glex1'][index] = sum

# dict 1
for index, row in test_df.iterrows():
    sum = 0
    text = row["req-text-clean"].split()
    for w in text:
      if w in coef2.index.tolist():
        sum += coef2[1][w]*tf_idf_test[w][index]
    sum = sum + svc.intercept_[1]
    test_df[f'glex2'][index] = sum

# dict 1
for index, row in test_df.iterrows():
    sum = 0
    text = row["req-text-clean"].split()
    for w in text:
      if w in coef3.index.tolist():
        sum += coef3[2][w]*tf_idf_test[w][index]
    sum = sum + svc.intercept_[2]
    test_df[f'glex3'][index] = sum

test_df.groupby(['Age'])[['glex1', 'glex2', 'glex3']].describe().T

/tmp/ipykernel_14084/2520854140.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[f'glex1'][index] = sum
/tmp/ipykernel_14084/2520854140.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[f'glex2'][index] = sum
/tmp/ipykernel_14084/2520854140.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[f'glex3'][index] = sum


Age                    0            1            2
glex1 count  3354.000000  3396.000000  2803.000000
      mean      0.650282    -0.523065    -0.338565
      std       1.828683     1.955025     1.588826
      min     -23.206401   -14.370070   -13.128862
      25%      -0.254756    -1.318329    -1.152842
      50%       0.619001    -0.369372    -0.302123
      75%       1.374954     0.487224     0.526575
      max      25.749522    21.690244     8.239131
glex2 count  3354.000000  3396.000000  2803.000000
      mean      1.212777     0.524283    -0.453269
      std       2.011427     2.006953     1.672790
      min     -12.478504   -15.307135   -14.315920
      25%       0.146420    -0.499951    -1.294656
      50%       1.067732     0.456920    -0.480441
      75%       2.033702     1.516962     0.402311
      max      37.798964    22.135998     9.238547
glex3 count  3354.000000  3396.000000  2803.000000
      mean      0.850870     1.088488    -0.300805
      std       1.929163     2.157455     1.683721
      min     -10.765882   -16.530285   -11.981856
      25%      -0.096685    -0.048637    -1.106857
      50%       0.739261     0.896920    -0.332394
      75%       1.605425     1.855034     0.560650
      max      48.107057    34.751546    23.995681

In [92]:
from sklearn import metrics

In [93]:
test_df.head()

,Unnamed: 0,age-bracket,req-text,req-text-clean,Age,glex1,glex2,glex3,c,c50,c5025,cmean,c75,c7525
0,34599,a43-xx,Solicito os registro de reclamações feitas a e...,solicito registro reclamações feitas ouvidoria...,2,-0.312707,-0.057745,-1.074704,-1,-1,-1,-1,-1,-1
1,23357,a17-30,"No estado do Amazonas, existem agências da pre...",estado amazonas existem agências previdência s...,0,-0.153447,1.151447,1.144852,1,1,-1,1,-1,-1
2,31440,a17-30,"Olá, solicito informações sobre a quantidade d...",olá solicito informações sobre quantidade códi...,0,4.736929,2.013420,-0.103715,-1,-1,0,-1,-1,-1
3,15896,a17-30,"Boa tarde, Estou com alguns problemas para con...",boa tarde alguns problemas conseguir cidadania...,0,0.089116,0.573233,0.438379,1,-1,-1,-1,-1,-1
4,15489,a17-30,"Referente à demanda 80200000235201650, percebi...",referente demanda 80200000235201650 percebi al...,0,1.278547,0.836758,-1.104455,-1,-1,-1,-1,-1,-1


In [94]:
test_df['c'] = -1

for index, row in test_df.iterrows():
  if row['glex1'] > 0 and row['glex2'] > 0 and row['glex3'] > 0: # delimitaria só pelo glex da regra
    test_df['c'][index] = 0
  elif row['glex1'] < 0 and row['glex2'] > 0 and row['glex3'] > 0:
    test_df['c'][index] = 1
  elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < 0:
    test_df['c'][index] = 2

test_df_dict = test_df[test_df['c'] != -1]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['c'].to_list(), test_df_dict['Age'].to_list(), average='macro')}")

/tmp/ipykernel_14084/3367236266.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c'][index] = 2
/tmp/ipykernel_14084/3367236266.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c'][index] = 1
/tmp/ipykernel_14084/3367236266.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c'][index] = 0


F1 (dict): 0.5634012033010277


In [75]:
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.c!=-1].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.c!=-1].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.c!=-1) & (test_df.c==test_df.Age)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.c!=-1) & (test_df.c==test_df.Age)].shape[0]/test_df[test_df.c!=-1].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.c!=-1) & (test_df.c==test_df.Age)].shape[0]/test_df.shape[0]}')

Instâncias totais: 9553
Instâncias classificadas: 6402
Porcentagem classificadas: 0.6701559719459855
Instâncias classificadas corretamente: 3595
Porcentagem classificadas corretamente: 0.5615432677288348
Porcentagem classificadas corretamente das totais: 0.37632157437454206


In [95]:
df_stat = train_df.groupby(['Age'])[['glex1', 'glex2', 'glex3']].describe().T

In [96]:
test_df['c50'] = -1

for index, row in test_df.iterrows():
  if row['glex1'] > df_stat[0]['glex1']['50%'] and row['glex2'] > 0 and row['glex3'] > 0: # delimitaria só pelo glex da regra
    test_df['c50'][index] = 0
  elif row['glex1'] < 0 and row['glex2'] > df_stat[1]['glex2']['50%'] and row['glex3'] > 0:
    test_df['c50'][index] = 1
  elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < df_stat[2]['glex3']['50%']:
    test_df['c50'][index] = 2

test_df_dict = test_df[test_df['c50'] != -1]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['c50'].to_list(), test_df_dict['Age'].to_list(), average='macro')}")
print('')
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.c50!=-1].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.c50!=-1].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.c50!=-1) & (test_df.c50==test_df.Age)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.c50!=-1) & (test_df.c50==test_df.Age)].shape[0]/test_df[test_df.c50!=-1].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.c50!=-1) & (test_df.c50==test_df.Age)].shape[0]/test_df.shape[0]}')

/tmp/ipykernel_14084/3046944293.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c50'][index] = 2
/tmp/ipykernel_14084/3046944293.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c50'][index] = 1
/tmp/ipykernel_14084/3046944293.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c50'][index] = 0


F1 (dict): 0.6126143766416132

Instâncias totais: 9553
Instâncias classificadas: 4438
Porcentagem classificadas: 0.4645661048885167
Instâncias classificadas corretamente: 2727
Porcentagem classificadas corretamente: 0.6144659756647138
Porcentagem classificadas corretamente das totais: 0.28546006490107817


In [97]:
test_df_class = test_df[test_df.c50!=-1]

In [98]:
test_df_class.shape

(4438, 14)

In [99]:
test_df_class.to_csv('../data/dict-class50.csv', encoding='latin-1')

In [78]:
test_df['c5025'] = -1

for index, row in test_df.iterrows():
  if row['glex1'] > df_stat[0]['glex1']['50%'] and row['glex2'] > df_stat[0]['glex2']['25%'] and row['glex3'] > df_stat[0]['glex3']['25%']: # delimitaria só pelo glex da regra
    test_df['c5025'][index] = 0
  elif row['glex1'] < df_stat[1]['glex1']['25%'] and row['glex2'] > df_stat[1]['glex2']['50%'] and row['glex3'] > df_stat[1]['glex3']['25%']:
    test_df['c5025'][index] = 1
  elif row['glex1'] < df_stat[2]['glex1']['25%'] and row['glex2'] < df_stat[2]['glex2']['25%'] and row['glex3'] < df_stat[2]['glex3']['50%']:
    test_df['c5025'][index] = 2

test_df_dict = test_df[test_df['c5025'] != -1]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['c5025'].to_list(), test_df_dict['Age'].to_list(), average='macro')}")
print('')
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.c5025!=-1].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.c5025!=-1].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.c5025!=-1) & (test_df.c5025==test_df.Age)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.c5025!=-1) & (test_df.c5025==test_df.Age)].shape[0]/test_df[test_df.c5025!=-1].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.c5025!=-1) & (test_df.c5025==test_df.Age)].shape[0]/test_df.shape[0]}')

/tmp/ipykernel_14084/2367062825.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c5025'][index] = 0
/tmp/ipykernel_14084/2367062825.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c5025'][index] = 2
/tmp/ipykernel_14084/2367062825.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c5025'][index] = 1


F1 (dict): 0.5673198570978709

Instâncias totais: 9553
Instâncias classificadas: 2640
Porcentagem classificadas: 0.2763529781220559
Instâncias classificadas corretamente: 1677
Porcentagem classificadas corretamente: 0.6352272727272728
Porcentagem classificadas corretamente das totais: 0.17554694860253323


In [203]:
test_df_class = test_df[test_df.c5025!=-1]

In [204]:
test_df_class.shape

(2751, 13)

In [205]:
test_df_class.to_csv('../data/dict-class.csv', encoding='latin-1')

In [79]:
test_df['cmean'] = -1

for index, row in test_df.iterrows():
  if row['glex1'] > df_stat[0]['glex1']['mean'] and row['glex2'] > 0 and row['glex3'] > 0: # delimitaria só pelo glex da regra
    test_df['cmean'][index] = 0
  elif row['glex1'] < 0 and row['glex2'] > df_stat[1]['glex2']['mean'] and row['glex3'] > 0:
    test_df['cmean'][index] = 1
  elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < df_stat[2]['glex3']['mean']:
    test_df['cmean'][index] = 2


test_df_dict = test_df[test_df['cmean'] != -1]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['cmean'].to_list(), test_df_dict['Age'].to_list(), average='macro')}")
print('')
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.cmean!=-1].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.cmean!=-1].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.cmean!=-1) & (test_df.cmean==test_df.Age)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.cmean!=-1) & (test_df.cmean==test_df.Age)].shape[0]/test_df[test_df.cmean!=-1].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.cmean!=-1) & (test_df.cmean==test_df.Age)].shape[0]/test_df.shape[0]}')

/tmp/ipykernel_14084/2304313386.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cmean'][index] = 1
/tmp/ipykernel_14084/2304313386.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cmean'][index] = 0
/tmp/ipykernel_14084/2304313386.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cmean'][index] = 2


F1 (dict): 0.6021839951722482

Instâncias totais: 9553
Instâncias classificadas: 4310
Porcentagem classificadas: 0.45116717261593214
Instâncias classificadas corretamente: 2605
Porcentagem classificadas corretamente: 0.6044083526682135
Porcentagem classificadas corretamente das totais: 0.2726892075787711


In [80]:
test_df['c75'] = -1

for index, row in test_df.iterrows():
  if row['glex1'] > df_stat[0]['glex1']['75%'] and row['glex2'] > 0 and row['glex3'] > 0: # delimitaria só pelo glex da regra
    test_df['c75'][index] = 0
  elif row['glex1'] < 0 and row['glex2'] > df_stat[1]['glex2']['75%'] and row['glex3'] > 0:
    test_df['c75'][index] = 1
  elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < df_stat[2]['glex3']['75%']:
    test_df['c75'][index] = 2

test_df_dict = test_df[test_df['c75'] != -1]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['c75'].to_list(), test_df_dict['Age'].to_list(), average='macro')}")
print('')
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.c75!=-1].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.c75!=-1].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.c75!=-1) & (test_df.c75==test_df.Age)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.c75!=-1) & (test_df.c75==test_df.Age)].shape[0]/test_df[test_df.c75!=-1].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.c75!=-1) & (test_df.c75==test_df.Age)].shape[0]/test_df.shape[0]}')

/tmp/ipykernel_14084/3500725655.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c75'][index] = 0
/tmp/ipykernel_14084/3500725655.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c75'][index] = 2
/tmp/ipykernel_14084/3500725655.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c75'][index] = 1


F1 (dict): 0.5422717071102133

Instâncias totais: 9553
Instâncias classificadas: 3716
Porcentagem classificadas: 0.38898775253846957
Instâncias classificadas corretamente: 2128
Porcentagem classificadas corretamente: 0.5726587728740581
Porcentagem classificadas corretamente das totais: 0.22275724903171779


In [63]:
test_df['c7525'] = -1

for index, row in test_df.iterrows():
  if row['glex1'] > df_stat[0]['glex1']['75%'] and row['glex2'] > df_stat[0]['glex2']['25%'] and row['glex3'] > df_stat[0]['glex3']['25%']: # delimitaria só pelo glex da regra
    test_df['c7525'][index] = 0
  elif row['glex1'] < df_stat[1]['glex1']['25%'] and row['glex2'] > df_stat[1]['glex2']['75%'] and row['glex3'] > df_stat[1]['glex3']['25%']:
    test_df['c7525'][index] = 1
  elif row['glex1'] < df_stat[2]['glex1']['25%'] and row['glex2'] < df_stat[2]['glex2']['25%'] and row['glex3'] < df_stat[2]['glex3']['75%']:
    test_df['c7525'][index] = 2

test_df_dict = test_df[test_df['c7525'] != -1]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['c7525'].to_list(), test_df_dict['Age'].to_list(), average='macro')}")
print('')
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.c7525!=-1].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.c7525!=-1].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.c7525!=-1) & (test_df.c7525==test_df.Age)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.c7525!=-1) & (test_df.c7525==test_df.Age)].shape[0]/test_df[test_df.c7525!=-1].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.c7525!=-1) & (test_df.c7525==test_df.Age)].shape[0]/test_df.shape[0]}')

/tmp/ipykernel_14084/446884030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c7525'][index] = 0
/tmp/ipykernel_14084/446884030.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c7525'][index] = 2
/tmp/ipykernel_14084/446884030.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c7525'][index] = 1


F1 (dict): 0.6053250097094934

Instâncias totais: 9553
Instâncias classificadas: 1615
Porcentagem classificadas: 0.1690568407830001
Instâncias classificadas corretamente: 1043
Porcentagem classificadas corretamente: 0.6458204334365325
Porcentagem classificadas corretamente das totais: 0.109180362189888
